In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d


from transformer_IDAO import TransformerIDAO
from clustering.clustering import Clustering2D
from prediction import Model2D
import importlib

%matplotlib inline 

In [2]:
#full_test = pd.read_csv("./data/test.csv", index_col='id')
#full_train = pd.read_csv("./data/train.csv", index_col='id')
full_test = pd.read_csv("../../data/test.csv", index_col='id')
full_train = pd.read_csv("../../data/train.csv", index_col='id')

transformer = TransformerIDAO()

In [3]:
transformer.fit_transform(full_train)
print(full_train.columns)

C:\Users\79242\Documents\Sandbox\IDAO\opeJupu\idao_oski\transformer_IDAO.py:35: RuntimeWarning: invalid value encountered in true_divide
  a /= norm
C:\Users\79242\Documents\Sandbox\IDAO\opeJupu\idao_oski\transformer_IDAO.py:36: RuntimeWarning: invalid value encountered in true_divide
  b /= norm
C:\Users\79242\Documents\Sandbox\IDAO\opeJupu\idao_oski\transformer_IDAO.py:37: RuntimeWarning: invalid value encountered in true_divide
  c /= norm


ROTATE
Index(['epoch', 'sat_id', 'x', 'y', 'z', 'Vx', 'Vy', 'Vz', 'x_sim', 'y_sim',
       'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim', 'x_transformed', 'y_transformed',
       'z_transformed', 'Vx_transformed', 'Vy_transformed', 'Vz_transformed'],
      dtype='object')


In [4]:
clustering1 = Clustering2D('KMeans', 15)
clusters_map = clustering1.fit_predict(transformer.train_coefs)

In [5]:
model = Model2D()
new_test = model.predict(full_train.copy(), full_test.copy(), clusters_map)

COORDS HAS ENUMERATED


In [6]:
new_test['z_pred'] = 0.0
new_test['Vz_pred'] = 0.0
new_test = transformer.inv_transform(new_test)

In [10]:
full_test = new_test

In [15]:
# отрисовка нужного рядя для анализа
import numpy as np
import pandas as pd
import lightgbm as lgbm
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import time
from sklearn.linear_model import LinearRegression
from tqdm import tqdm_notebook as tqdm


figure = plt.figure(figsize=(10, 5))


sat_id = 32
real_col_name = 'x'
sim_col_name = real_col_name + '_sim'
pred_col_name = real_col_name + '_pred_transformed'

fig = go.Figure()
fig.add_trace(go.Scatter(x=full_test[full_test.sat_id == sat_id].epoch,
                             y=full_test[full_test.sat_id == sat_id][pred_col_name],
                             name="Estimated " + real_col_name,
                             ))

# fig.add_trace(go.Scatter(x=full_test[full_test.sat_id == sat_id].epoch,
#                          y=full_test[full_test.sat_id == sat_id][sim_col_name],
#                          name="Simulation " + real_col_name,
#                          ))

# fig.add_trace(go.Scatter(x=full_test[full_test.sat_id == sat_id].epoch,
#                          y=full_test[full_test.sat_id == sat_id][sim_col_name],
#                          name="Sim_test " + real_col_name,
#                          ))
fig.add_trace(go.Scatter(x=full_train[full_train.sat_id == sat_id].epoch,
                         y=full_train[full_train.sat_id == sat_id][real_col_name],
                         name="Real " + real_col_name,
                         ))

# fig.add_trace(go.Scatter(x=full_train[full_train.sat_id == sat_id].epoch,
#                          y=full_train[full_train.sat_id == sat_id][sim_col_name],
#                          name="Simulation " + real_col_name,
#                          ))

fig.update_layout(title_text='Time Series for ' + real_col_name,
                  xaxis_rangeslider_visible=True,
                  yaxis_title=real_col_name)
fig.show()

<Figure size 720x360 with 0 Axes>

In [7]:
WIDTH = 4
width = 24 * WIDTH

full_train['coord_num'] = 0
full_train['vel_num'] = 0
full_test['coord_num'] = 0
full_test['vel_num'] = 0

UNIQ = full_test.sat_id.unique()

for sat_id in full_train.sat_id.unique():
    # enumerating coords
    df_train = full_train[full_train.sat_id == sat_id]
    if sat_id in UNIQ:
        df_test = full_test[full_test.sat_id == sat_id]
    df = df_train.iloc[-width:]
    for trans_col, num_col in zip(['x_transformed', 'Vx_transformed'], ['coord_num', 'vel_num']):
        idx_min  = df[trans_col].idxmin()
        idx_last = (len(df_train) + idx_min) % 24
        full_train.loc[df_train.index, num_col] = np.fromfunction(lambda i: (i + idx_min) % 24,
                                                                  (len(df_train), ), dtype=np.int16)
        if sat_id in UNIQ:
            full_test.loc[df_test.index, num_col] = np.fromfunction(lambda i: (i + idx_last) % 24,
                                                                    (len(df_test), ), dtype=np.int16)

In [10]:
full_test['x_pred'] = 0.0; full_test['y_pred'] = 0.0
full_test['Vx_pred'] = 0.0; full_test['Vy_pred'] = 0.0

for cluster in set(clusters_map.values()):
    for trans_cols, pred_cols, num_col in zip([['x_transformed', 'y_transformed'],
                                               ['Vx_transformed', 'Vy_transformed']],
                                              [['x_pred', 'y_pred'],
                                               ['Vx_pred', 'Vy_pred']],
                                              ['coord_num', 'vel_num']):
        df_train = full_train[full_train.cluster == cluster]
        df_test = full_test[full_test.cluster == cluster]
        df_train = df_train[df_train.sat_id.isin(df_test.sat_id.unique())]

        for i in tqdm(range(24), leave=False):
            X_train = np.arange(WIDTH, dtype=np.float64)
            per_train = df_train[df_train[num_col] == i]
            per_test = df_test[df_test[num_col] == i]
            y_train = np.zeros(WIDTH, dtype=np.float64)
            y_train_cnt = np.zeros(WIDTH, dtype=np.float64)

            # select the last width of observations for every sat_id
            for col, pred_col in zip(trans_cols, pred_cols):
                linear_reg = LinearRegression()
                for j in range(-1, -WIDTH, -1):
                    for sat_id in per_test.sat_id.unique():
                        sat_col_val = per_train.loc[(per_train.sat_id == sat_id).index, col].values
                        if abs(j) <= len(sat_col_val):
                            y_train[j] += sat_col_val[j]
                            y_train_cnt[j] += 1

                # mean aggregation
                y_train_cnt[y_train_cnt == 0] = 1
                y_train /= y_train_cnt

                linear_reg = LinearRegression()
                linear_reg.fit(X_train.reshape(-1, 1), y_train)

                for sat_id in per_test.sat_id.unique():
                    X_test = np.arange(len(per_test)) + WIDTH
                    y_pred = linear_reg.predict(X_test.reshape(-1, 1))
                    full_test.loc[(per_test.sat_id == sat_id).index, pred_col] = y_pred

In [11]:
full_test

,sat_id,epoch,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim,coord_num,vel_num,cluster,x_pred,y_pred,Vx_pred,Vy_pred
id,,,,,,,,,,,,,,,
3927,1,2014-02-01T00:01:45.162,-13366.891347,-14236.753503,6386.774555,4.333815,-0.692764,0.810774,3,19,2,-2.443935e+04,1.461184e+04,2.085128e+04,-286.586602
3928,1,2014-02-01T00:22:57.007,-7370.434039,-14498.771520,7130.411325,5.077413,0.360609,0.313402,4,20,2,-1.701281e+04,1.652302e+04,2.085198e+04,-287.130398
3929,1,2014-02-01T00:44:08.852,-572.068654,-13065.289498,7033.794876,5.519106,2.012830,-0.539412,5,21,2,-8.186814e+03,1.780525e+04,2.085254e+04,-287.925498
3930,1,2014-02-01T01:05:20.697,6208.945257,-9076.852425,5548.296900,4.849212,4.338955,-1.869600,6,22,2,2.028799e+03,1.820480e+04,2.085259e+04,-289.114320
3931,1,2014-02-01T01:26:32.542,10768.200284,-2199.706707,2272.014862,1.940505,6.192887,-3.167724,7,23,2,1.349064e+04,1.733232e+04,2.085108e+04,-290.748326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231060,597,2014-02-28T05:19:01.386,28595.031634,-85458.623976,5021.767767,-1.721131,-0.011611,0.122357,18,9,8,-6.166618e+06,1.070079e+06,2.950210e+06,15610.027174
1231061,597,2014-02-28T07:21:46.454,15547.173728,-84233.509948,5840.616702,-1.815323,0.353445,0.098927,19,10,8,-5.580770e+06,1.582812e+06,2.959263e+06,15653.228246
1231062,597,2014-02-28T09:24:31.522,1981.107111,-80123.860051,6458.394351,-1.858963,0.773846,0.067379,20,11,8,-4.768602e+06,2.053745e+06,2.977338e+06,15741.780440
